In [4]:
'''
Solve the Travelling Salesman Problem using local search

Each state is a list indicating the list of cities in order of travelling.
Each action is a swap of the two cities.

The simulated annealing algorithm works better than the hill-climbing algorithm.
The hill-climbing algorithm can only decrease its total trip length, meaning that
it will not take a longer solution at any point, even if such a step back would
result in a shorter path in the end.  It is stuck on the hill on which it starts.
The simulated annealing algorithm, however, occasionally and randomly takes a jump that 
increases the length of the solution.  However, this jump could lead to a hill with a higher peak 
than the starting hill, improving the overall solution.  Granted, this can mean that the simulated 
annealing algorithm actually takes a step back and gives a longer solution than the hill-climbing               
algorithm, but such behavior is not typical.
'''
from tools.aima.search import Problem, hill_climbing, simulated_annealing, \
    exp_schedule, genetic_search
from timeit import default_timer as timer


class TSP(Problem):
    '''
    State: A list of the cities in visitation order
    Action: Swap two cities
    '''

    def __init__(self, initial, lengths):
        self.initial = initial
        self.lengths = lengths

    def actions(self, state):
        actions = []
        for i in range(len(state)-1):
            actions.append([state[i+1], state[i]])
        return actions

    # There is no goal_test for this problem.

    def result(self, state, action):
        newState = state[:]
        # find indices
        index1 = newState.index(action[1])
        index2 = newState.index(action[0])

        # remove old states
        newState.remove(action[1])
        newState.remove(action[0])

        # insert new states
        newState.insert(index1, action[0])
        newState.insert(index2, action[1])
        return newState

    def value(self, state):
        tripLength = 0
        for i in range(len(state)-1):
            letter1 = state[i]
            letter2 = state[i+1]

            if letter1 > letter2:
                key = letter2 + letter1
            else:
                key = letter1 + letter2

            tripLength += self.lengths[key]

        return -tripLength


# This creates an initial list of cities and the distances between them
lengths = {"AB":1, "AC":11, "AD":18, "AE":20, "AF":6, "AG":6, "AH":3, "AI":2, "AJ":15,
           "BC":1, "BD":9, "BE":7, "BF":20, "BG":1, "BH":17, "BI":13, "BJ":5,
           "CD":20, "CE":15, "CF":7, "CG":5, "CH":19, "CI":19, "CJ":1,
           "DE":3, "DF":14, "DG":20, "DH":13, "DI":7, "DJ":2,
           "EF":20, "EG":4, "EH":8, "EI":3, "EJ":5,
           "FG":18, "FH":8, "FI":12, "FJ":13,
           "GH":2, "GI":1, "GJ":1,
           "HI":14, "HJ":15,
           "IJ":18}
initial = ["D", "A", "C", "B", "E", "F", "G", "H", "I", "J"]
p = TSP(initial, lengths)  # Create an Abs variant problem, specifying the delta step value.
print('Initial                      x: ' + str(p.initial) + '\t\tvalue: ' + str(p.value(initial)))

startH = timer()
# Solve the problem using hill-climbing.
hill_solution = hill_climbing(p)
print('Hill-climbing solution       x: ' + str(hill_solution) + '\tvalue: ' + str(p.value(hill_solution)))
endH = timer()

startS = timer()
# Solve the problem using simulated annealing.
annealing_solution = simulated_annealing(p, exp_schedule(k=20, lam=0.005, limit=1000))
print('Simulated annealing solution x: ' + str(annealing_solution) + '\tvalue: ' + str(p.value(annealing_solution)))
endS = timer()

print("Hill Time: %(time)f", {'time':(endH - startH)})
print("Annealing Time: %(time)f", {'time':(endS - startS)})

Initial                      x: ['D', 'A', 'C', 'B', 'E', 'F', 'G', 'H', 'I', 'J']		value: -109
Hill-climbing solution       x: ['D', 'A', 'B', 'C', 'E', 'H', 'F', 'I', 'G', 'J']	value: -65
Simulated annealing solution x: ['C', 'B', 'D', 'J', 'G', 'I', 'A', 'F', 'H', 'E']	value: -38
Hill Time: %(time)f {'time': 0.0007190010001068003}
Annealing Time: %(time)f {'time': 0.03184835499996552}


In [1]:
'''
Create a mini schedule for the CS department to use for access to SB354 and SB372

The classes were chosen as the variable because they are the board on which this problem is 
solved.  Because they were the element with the most possibilities, they only appear once in
in the solution.  The professors, times, and rooms, on the other hand, each appear multiple        
times.
 
The possible domains for this problem was every combination of professor, room and       
time with each combination rendered as a tuple.  This allowed access to each field 
throughout the process of finding a solution and kept the possible options together
well.

The constraints chosen for this problem were:
each course offered once by one faculty member
each faculty member teaches only one thing at a time
only one class can occupy a room at a time
no teacher may teach two classes in a row (added to make schedule equitable)
'''

from csp import CSP, min_conflicts, backtracking_search, AC3, parse_neighbors
from search import depth_first_graph_search

# Schedule Problem Class
def Schedule():
    Profs = 'dschuurman adams vnorman kvlinden'.split()
    Classes = 'cs108 cs112 cs212 cs214 cs232 cs300 cs344'.split()
    Times = 'mwf800 mwf900 mwf1030 mwf1130'.split()
    Rooms = 'sb354 sb372'.split()
    variables = Classes
    domains = {}
    for var in variables:
        domains[var] = []
        for Prof in Profs:
            for Time in Times:
                for Room in Rooms:
                    domains[var].append((Prof, Time, Room))
    neighbors = parse_neighbors('cs108: ; cs232: ; cs112: ; cs212: ; cs214: ; cs300: ; cs344: ', variables)
    for type in [Classes]:
        for A in type:
            for B in type:
                if A != B:
                    if B not in neighbors[A]:
                        neighbors[A].append(B)
                    if A not in neighbors[B]:
                        neighbors[B].append(A)

    def schedule_constraints(A, a, B, b, recurse=0):
        same = (a == b)
        if A == "cs232" and a[0] != "vnorman":
            return False
        if A == "cs344" and a[0] != "kvlinden":
            return False
        if A == "cs112" and a[0] != "adams":
            return False
        if A == "cs300" and a[0] != "dschuurman":
            return False
        if recurse == 0:
            schedule_constraints(B, b, A, a, recurse=1)
        if A == B:
            return same
        if A != B:
            # if same professor
            if a[0] == b[0]:
                # if meeting at same time
                if a[1] == b[1]:
                    return False
                # No professor can teach consecutive classes
                elif a[1] == "mwf800" and b[1] == "mwf900":
                    return False
                elif b[1] == "mwf800" and a[1] == "mwf900":
                    return False
                elif a[1] == "mwf1030" and b[1] == "mwf900":
                    return False
                elif b[1] == "mwf1030" and a[1] == "mwf900":
                    return False
                elif a[1] == "mwf1030" and b[1] == "mwf1130":
                    return False
                elif b[1] == "mwf1030" and a[1] == "mwf1130":
                    return False
                else:
                    return True
            # if same time
            if a[1] == b[1]:
                # if classes have same professor or room
                if a[0] == b[0] or a[2] == b[2]:
                    return False
                else:
                    return True
            if a[2] == b[2]:
                if a[1] == b[1]:
                    return False
                else:
                    return True
            else:
                return True

        raise Exception('error')
    return CSP(variables, domains, neighbors, schedule_constraints)


def print_solution(result):
    """A CSP solution printer copied from csp.py."""
    for Class in 'cs108 cs112 cs212 cs214 cs232 cs300 cs344'.split():
        print('Class: ', Class)
        output = result[Class]
        print("Professor: ", output[0])
        print("Timeslot: ", output[1])
        print("Room: ", output[2])
        print("\n")


puzzle = Schedule()

# result = depth_first_graph_search(puzzle)
# result = AC3(puzzle)
result = backtracking_search(puzzle)
# result = min_conflicts(puzzle, max_steps=1000)

if puzzle.goal_test(puzzle.infer_assignment()):
    print("Solution:\n")
    print_solution(result)
else:
    print("failed...")
    print(puzzle.curr_domains)
    puzzle.display(puzzle.infer_assignment())

Solution:

Class:  cs108
Professor:  dschuurman
Timeslot:  mwf800
Room:  sb354


Class:  cs112
Professor:  adams
Timeslot:  mwf800
Room:  sb372


Class:  cs212
Professor:  dschuurman
Timeslot:  mwf1030
Room:  sb354


Class:  cs214
Professor:  adams
Timeslot:  mwf1030
Room:  sb372


Class:  cs232
Professor:  vnorman
Timeslot:  mwf900
Room:  sb354


Class:  cs300
Professor:  vnorman
Timeslot:  mwf1130
Room:  sb354


Class:  cs344
Professor:  kvlinden
Timeslot:  mwf900
Room:  sb372


